In [6]:
#!/usr/bin/env python3
"""
Module Docstring
"""

__author__ = "Your Name"
__version__ = "0.1.0"
__license__ = "MIT"

import argparse
import os
from logzero import logger
from espn_api.football import League


POSITIONS = ["QB", "WR", "TE", "RB"]

""" Main entry point of the app """
logger.info("hello world from the notebook")
# logger.info(args)


# Get environment variables
swid = os.getenv('SWID')
espn_s2 = os.environ.get('ESPN_S2')

if swid == None or espn_s2 == None:
    print("SWID or ESPN_S2 environment variables empty. Exiting...")



## Get Box Scores, pass in week 
week = 2
# if args.week != None:
#     week = args.week
# for week in range(week):
league = League(league_id=761056, year=2022, espn_s2=espn_s2,swid=swid)

print("Week: ", week)
box_scores = league.box_scores(2)
for matchup in box_scores:
    print(matchup.home_team.team_name )

[I 220925 01:43:25 2764710387:19] hello world from the notebook


Week:  2
J.G.'s  ZZ's 
Chiraqui Soldiers
Top G
Begelbaum Donuts
Kupp My  Bells Gurley


In [7]:
def biggestBlowOut(league, week):
    outcomes = []
    
    box_scores = league.box_scores(week)
    
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        diff = 0
        if matchup.home_score >= matchup.away_score:
            diff = matchup.home_score - matchup.away_score
            outcomes.append((matchup, diff))
        else:
            diff = matchup.away_score - matchup.home_score
            outcomes.append((matchup, diff))
            
    sorted_scores = sorted(outcomes, key=lambda tup: tup[1], reverse=True)

    return sorted_scores


blowoutMatchup = biggestBlowOut(league, week)[0]

print("======== Biggest Blowout (", blowoutMatchup[1], ") =========")
print(blowoutMatchup[0].home_team.team_name, blowoutMatchup[0].home_score, " vs ", blowoutMatchup[0].away_team.team_name, blowoutMatchup[0].away_score )


======== Biggest Blowout ( 42.7 ) =========
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


In [9]:
def biggestBenchWarmer(league, week, position):
    if position not in POSITIONS:
        return []
    
    
    benchWarmers = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        ## For Each Matchip calculate the diff
        for player in matchup.home_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.home_team, " ", player.points)
                benchWarmers.append((player.points,player.name,player.position,matchup.home_team.team_name))
                
                
        for player in matchup.away_lineup:
            if player.position == position and player.slot_position == "BE":
#                 print("player: ", player.name, " position: ", player.position, " ", player.slot_position, " ", matchup.away_team, " ",player.points )
                benchWarmers.append((player.points,player.name,player.position,matchup.away_team.team_name))
    

    return sorted(benchWarmers, key=lambda tup: tup[0], reverse=True)


def prettyPrintBenchWarmers(benchWarmers, position):
    print("=============== ", position, " ===============")
    for index, warmer in enumerate(benchWarmers):
        print(index+1,"", warmer[0], " ", warmer[1], " ", warmer[3])
    print("  ")
        

### QBs
list_len = 3
qbWarmers = biggestBenchWarmer(league, week, "QB")
prettyPrintBenchWarmers(qbWarmers[:list_len], "QB")

### RBs
rbWarmers = biggestBenchWarmer(league, week, "RB")
prettyPrintBenchWarmers(rbWarmers[:list_len], "RB")

### TEs
teWarmers = biggestBenchWarmer(league, week, "TE")
prettyPrintBenchWarmers(teWarmers[:list_len], "TE")


#### WRs
teWarmers = biggestBenchWarmer(league, week, "WR")
prettyPrintBenchWarmers(teWarmers[:list_len], "WR")
        
    

===============  QB  ===============
1  26.7   Carson Wentz   Gisele'd in my  Wentz
2  14.4   Marcus Mariota   Rosen Sandler
3  12.6   Mac Jones   The  Red Army
  
===============  RB  ===============
1  17.8   Tony Pollard   Top G
2  15.7   Damien Harris   Chiraqui Soldiers
3  14.6   David Montgomery   Kupp My  Bells Gurley
  
===============  TE  ===============
1  14.0   Darren Waller   Gisele'd in my  Wentz
2  10.7   Dallas Goedert   J.G.'s  ZZ's 
3  4.1   T.J. Hockenson   Gisele'd in my  Wentz
  
===============  WR  ===============
1  34.9   Amon-Ra St. Brown   Rosen Sandler
2  26.0   Garrett Wilson   J.G.'s  ZZ's 
3  20.6   Drake London   Chiraqui Soldiers
  


In [67]:
# ### Total Points Played / Optimal Lineup
# lineup_format = {
#     "QB": 2
#     "RB": 2
#     "WR TE": 3
#     "FLEX": 1
# }
# # def optimalQB(lineup, position, num_starters):

    
#     for player in lineup:
#         # Add to starters
#         if player.position == position and player.slot_position != "IR" and player.slot_position != "BE":
#             print("Started: ", player.name)
#             starters.append(player) 
#             starter_ouput += player.points
#         ## Add All bench of similar position
#         if player.position == position and player.slot_position != "IR":
#             all_players.append(player)

#     ## Find Best Possible Output
#     for player in all_players:
#             # sort list by `points
#             all_players.sort(key=lambda x: x.points, reverse=True)
            
#     print("starter_ouput: ", starter_ouput)
#     print("all_players: ", all_players )
#     print("best_possible", sum(player.points for player in all_players[:num_starters]))
            

# # def managerEffencieny(league, week):
def optimalLineup(lineup):  

    QB = 2
    RB = 2
    WR_TE = 3
    FLEX = 1

    starters = []
    starter_ouput = 0
    all_players = []
    
    optimal_players = []
    optimal_output = 0
    
    for player in lineup:
        # Add to starters
        if player.slot_position != "IR" and player.slot_position != "BE":
#             print("Started: ", player.name)
            starters.append(player) 
            starter_ouput += player.points

#     print("lineup", lineup)
    all_players = sorted(lineup, key=lambda x: x.points, reverse=True)


    for player in all_players:
        if QB > 0 and player.position == "QB":
                QB = QB - 1
#                 print("Best QB: ", player.name )
                optimal_players.append(player)
                optimal_output += player.points

            
    ## Get Best FLEX Score
    for player in all_players:
        if (player.position == "RB" or player.position == "WR" or player.position == "TE") and FLEX > 0:
            FLEX = FLEX - 1
            optimal_players.append(player)
            optimal_output += player.points
     
    for player in all_players:
        if (player.position == "RB") and RB > 0 and (player not in optimal_players):
            RB = RB -1
            optimal_players.append(player)
            optimal_output += player.points
                
    for player in all_players:
        if (player.position == "TE" or player.position == "WR") and (player not in optimal_players) and  WR_TE > 0:
            WR_TE = WR_TE - 1
            optimal_players.append(player)
            optimal_output += player.points
    
    


#     print("starter_ouput: ", starter_ouput)
#     print("optimal_output: ", optimal_output, optimal_players)
    return starter_ouput, optimal_output
  

## Get Mananger Effecieny for Each Week
for i in range(week+1):
    if week == 0:
        continue
    print("Week: ", i+1)
    for matchup in league.box_scores(i+1):
            print("matchup", matchup)
            starter_ouput,optimal_output = optimalLineup(matchup.home_lineup)
            manager_eff.append((matchup.home_team.team_name,starter_ouput,optimal_output,starter_ouput/optimal_output))

            starter_ouput,optimal_output = optimalLineup(matchup.away_lineup)
            manager_eff.append((matchup.away_team.team_name,starter_ouput,optimal_output, starter_ouput/optimal_output))
            manager_eff = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=True)


def prettyPrintManagerEff(manager_eff):
    manager_eff_sorted = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=True)
    print("============== Best Manager efficiency =========")
    for manager in manager_eff_sorted[:2]:
        print("{0}: {1:.2f}/{2:.2f}  {3:4.2f}%".format(manager[0], manager[1],manager[2],manager[3]))
        
    
    worst_manager_eff_sorted = sorted(manager_eff, key=lambda manager_eff: manager_eff[3], reverse=False)
    print("============== Worst Manager efficiency =========")
    for manager in worst_manager_eff_sorted[:2]:
        print("{0}: {1:.2f}/{2:.2f}  {3:4.2f}%".format(manager[0], manager[1],manager[2],manager[3]*100))

prettyPrintManagerEff(manager_eff)

Week:  1
matchup Box Score(Team(Kupp My  Bells Gurley) at Team(Rosen Sandler))
matchup Box Score(Team(J.G.'s  ZZ's ) at Team(Top G))
matchup Box Score(Team(Gisele'd in my  Wentz) at Team(Grayson BeatYu))
matchup Box Score(Team(The  Red Army) at Team(El Chapo))
matchup Box Score(Team(Chiraqui Soldiers) at Team(Begelbaum Donuts))
Week:  2
matchup Box Score(Team(The  Red Army) at Team(J.G.'s  ZZ's ))
matchup Box Score(Team(Grayson BeatYu) at Team(Chiraqui Soldiers))
matchup Box Score(Team(Gisele'd in my  Wentz) at Team(Top G))
matchup Box Score(Team(Rosen Sandler) at Team(Begelbaum Donuts))
matchup Box Score(Team(El Chapo) at Team(Kupp My  Bells Gurley))
Week:  3
matchup Box Score(Team(Gisele'd in my  Wentz) at Team(The  Red Army))
matchup Box Score(Team(Begelbaum Donuts) at Team(Top G))


ZeroDivisionError: division by zero

In [34]:
def worstWin(league, week):
    box_scores = league.box_scores(week)
    lowestWinningScore = 99999999999
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            lowestWinningScore = min(matchup.home_score, lowestWinningScore)
        else:
            lowestWinningScore = min(matchup.away_score, lowestWinningScore)
            
        
        if lowestWinningScore == matchup.away_score or lowestWinningScore == matchup.home_score:
            saved = matchup
    print("================== Worst Win ===============")      
    print(saved.home_team.team_name, saved.home_score, " vs ", saved.away_team.team_name, saved.away_score)

    
worstWin(league, week)

================== Worst Win ===============
J.G.'s  ZZ's  89.8  vs  The  Red Army 83.6


In [35]:
def worstLoss(league, week):
    box_scores = league.box_scores(week)
    higestLoss = 0
    saved = None
    for matchup in box_scores:
        if matchup.home_score >= matchup.away_score:
            higestLoss = max(matchup.away_score, higestLoss)
        else:
            higestLoss = max(matchup.home_score, higestLoss)
            
        
        if higestLoss == matchup.away_score or higestLoss == matchup.home_score:
            saved = matchup
    print("================== Worst Loss ===============")
    print(saved.home_team.team_name, saved.home_score, " vs ", saved.away_team.team_name, saved.away_score)

    
worstLoss(league, week)

================== Worst Loss ===============
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


In [166]:
def startingLineupAverage(lineup):
    starterOutput = [] 
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            starterOutput.append(player.points)         
    return sum(starterOutput)/len(starterOutput)


def highestScoringStarter(lineup):
    highestScore = 0
    bestPlayer = None
    for player in lineup:
        if player.slot_position != "BE" and player.slot_position != "IR":
            if player.points > highestScore:
                highestScore = player.points
                bestPlayer = player
    return bestPlayer
    
    
## Team with greatest differences between max player and team average
def topHeavyTeams(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        maxPlayer = highestScoringStarter(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg, maxPlayer, maxPlayer.points - avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[3], reverse=True)
    return allTeamSorted
                                         

topHeavyList = topHeavyTeams(league, week)

print("======= Top Heavy Teams (Carried by one player) ========")
for index, team in enumerate(topHeavyList[:3]):
    print(index+1, " ", team[0], team[2].name, team[2].points, "{:.2f}".format(team[1]))
        

======= Top Heavy Teams (Carried by one player) ========
1   Rosen Sandler Tua Tagovailoa 38.7 12.12
2   Top G Stefon Diggs 38.8 13.07
3   El Chapo Lamar Jackson 42.5 20.24


In [169]:
## Team with greatest differences between max player and team average
def highestTeamAverageForStarters(league, week):
    allTeams = []
    box_scores = league.box_scores(week)
    for matchup in box_scores:
        
        ## HOME TEAM
        avg = startingLineupAverage(matchup.home_lineup)
        allTeams.append((matchup.home_team.team_name, avg))
        
        ## AWAY TEAM
        avg = startingLineupAverage(matchup.away_lineup)
        maxPlayer = highestScoringStarter(matchup.away_lineup)
        allTeams.append((matchup.away_team.team_name, avg))
        
    allTeamSorted = sorted(allTeams, key=lambda tup: tup[1], reverse=True)
    return allTeamSorted


print("======= All around hitters ( Highest team Average points)  ========")
for index, team in enumerate(highestTeamAverageForStarters(league,week)[:3]):
    print(index+1, " ", team[0], team[1])
                                   

======= All around hitters ( Highest team Average points)  ========
1   El Chapo 20.2375
2   Gisele'd in my  Wentz 16.1125
3   Grayson BeatYu 15.375


In [178]:
def standings(league):
    
    for week in range([1,2]):
        
        
    print(league.standings())
    print(league.power_rankings())
    
standings(league)

[Team(Grayson BeatYu), Team(Begelbaum Donuts), Team(J.G.'s  ZZ's ), Team(El Chapo), Team(Kupp My  Bells Gurley), Team(Gisele'd in my  Wentz), Team(The  Red Army), Team(Rosen Sandler), Team(Top G), Team(Chiraqui Soldiers)]
[('16.60', Team(Grayson BeatYu)), ('15.20', Team(El Chapo)), ('12.80', Team(J.G.'s  ZZ's )), ('12.65', Team(Begelbaum Donuts)), ('12.55', Team(Kupp My  Bells Gurley)), ('11.25', Team(The  Red Army)), ('10.55', Team(Gisele'd in my  Wentz)), ('10.50', Team(Rosen Sandler)), ('10.15', Team(Top G)), ('7.60', Team(Chiraqui Soldiers))]


#############################################
###########  Week 2 Output  #################
#############################################

######### Biggest Blowout ( 42.7 ) ####################
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


######## Biggest Bench Warmers (Highest Points left on bench ) ###########
===============  QB  ===============
1  26.7   Carson Wentz     Gisele'd in my  Wentz
2  14.4   Marcus Mariota   Rosen Sandler
3  12.6   Mac Jones   The  Red Army
4  10.3   Mitch Trubisky   J.G.'s  ZZ's 
5  7.6    Jameis Winston   Top G
  
===============  RB  ===============
1  17.8   Tony Pollard            Top G
2  15.7   Damien Harris           Chiraqui Soldiers
3  14.6   David Montgomery        Kupp My  Bells Gurley
4  12.5   Breece Hall             Top G
5  10.7   Darrell Henderson Jr.   El Chapo
  
===============  TE  ===============
1  14.0   Darren Waller   Gisele'd in my  Wentz
2  10.7   Dallas Goedert  J.G.'s  ZZ's 
3  4.1   T.J. Hockenson   Gisele'd in my  Wentz
4  2.8   Dalton Schultz   Begelbaum Donuts
5  0.0   George Kittle    El Chapo
  
=================================================

######### Worst Win (Lowest Scoring Winner) ################
J.G.'s  ZZ's  89.8  vs  The  Red Army 83.6

######### Worst Loss (Highest Scoring Loser) ################
Kupp My  Bells Gurley 119.2  vs  El Chapo 161.9


########## Top Heavy Teams ################
*: biggest difference between top performer and average starter output
     
     Team        |  Player  |   Top score  | Team Avg
1   Rosen Sandler  Tua Tagovailoa  38.7          12.12
2   Top G          Stefon Diggs    38.8          13.07
3   El Chapo Lamar Jackson         42.5          20.24



######## All around hitters ( Highest team Average points)  ################
1   El Chapo               20.2375
2   Gisele'd in my  Wentz  16.1125
3   Grayson BeatYu         15.375

